<a href="https://colab.research.google.com/github/johanhoffman/DD2363_VT22/blob/main/template-report-lab-X.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2: Matrix Factorization**
**Marc Hétier**

# **Abstract**

A short statement on who is the author of the file, and if the code is distributed under a certain license. 

In [ ]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2020 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [ ]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt

# **Introduction**

In this lab, we focus on matrix factorization through different problems, which involve the most basic operation in applied mathematic. First, we will implement a sparse format for matrices, and a sparse matrix-dense vector product (Problem 1). In Problem 2, 3 and 5, we interest on QR algorithm and it's use to solve linear problems or find eignevalues. On Problem 4, we implement a least square method to solve a linear problem, and finally in Problem 6 we look for a blocked matrix-matrix product.

Test will also be implemented, at least to verify accuracy and sometimes to test the performances.


# **Method**

## Problem 1 : Sparse matrix-vector product
We will first implement a class of sparse matrix, corresponding of CRS format. We add some methods to convert a numpy array into this format, and a print function.

In [ ]:
class Sparse:
    def __init__(self, size = (0,0), val = [], col=[], row=[]):
        self.size = size
        self.val = val
        self.col = col
        self.row_ptr = row

    def __str__(self) -> None:
        return "Value   : " + str(self.val) + '\n' + \
                "Col     : " + str(self.col)  + '\n' +  \
                "Row_ptr : " + str(self.row_ptr)
    
    def np_to_sp(self, A):
        self.val, self.col, self.row_ptr = [], [], []
        self.size = A.shape
        N, M = A.shape
        compt_row = 0
        for ind_row in range(0, N):
            test = 0
            for ind_col in range(0, M):
                value = A[ind_row, ind_col]
                if value != 0:
                    if not test:
                        for _ in range(compt_row+1):
                            self.row_ptr.append(len(self.val))
                        test = 1
                        compt_row = 0

                    self.val.append(value)
                    self.col.append(ind_col)
    
            if not test :
                compt_row += 1
        
        for _ in range(compt_row+1):
            self.row_ptr.append(len(self.val))
        return None

For example :

In [ ]:
A = np.array([[1,2],[0,3],[1,0]])
A_s = Sparse()
A_s.np_to_sp(A)
print("A : \n", A)
print("A_s :\n", A_s)

We can now implement the sparse-matrix vector product

In [ ]:
def sp_matrix_vector(A, b):
    """
    Input : a sparse matrix A, a numpy array b
    Output : numpy vector A*b
    """
    assert len(A.size) == 2, "A is not a matrix"
    assert len(b.shape) == 1, "b is not a vector"
    assert b.shape[0] == A.size[1], "dimension does not match"

    M = A.size[1]
    rslt = np.zeros_like(b)
    for i in range(M):
        for j in range(A.row_ptr[i], A.row_ptr[i+1]):
            rslt[i] += A.val[j] * b[A.col[j]]
        
    return rslt

## Problem 2 : QR factorization
In this problem, we implement a QR factorization of a real quadratic matrix A, using Gram-Schmidt procedure.
We first implement a subfunction which orthogonalize one vector versus a matrix, before using it on the main problem.

In [ ]:
def modified_GS(Q, a):
    """
    Input : an orthonormal matrix Q, a vector a
    Output : result of Gram-Schmidt procedure applied to (Q, a). A vector q, orthonormal
    with Q and a vector containing the coordinates of a in basis (Q, q)
    """
    N, J = Q.shape
    assert N==a.shape[0], "dimension does not match"
    q = deepcopy(a)
    coord = np.zeros(J+1)
    for j in range(J):
        ps = np.dot(Q[:,j], q)
        q = q - ps*Q[:,j]
        coord[j] = ps

    norm = np.linalg.norm(q)
    q = q/norm
    coord[J] = norm
    return q, coord

In [ ]:
def QR_decomp(A):
    """
    Input : a real square matrix A
    Output : a QR decomposition of A, using GS method
    """
    N, M = A.shape
    assert N==M, 'A is not a square matrix'
    Q = np.zeros((N,N))
    A_1 = A[:,0]
    norm_A_1 = np.linalg.norm(A_1)
    Q[:,0] = deepcopy(A_1)/norm_A_1
    R = np.zeros((N,N))
    R[0,0] = norm_A_1

    for ind in range(1, N):
        A_1 = A[:,ind]
        q, coord = modified_GS(Q[:,0:ind], A_1)
        Q[:,ind] = q
        R[0:ind+1, ind] = coord

    return Q, R

## Problem 3 : Direct solver Ax = b

In this problem, we have to solve a linear problem $Ax = b$ using a direct method. We will use the previous problem to solve this system by a clever way. Indeed, using the QR decomposition of $A = QR$, the system $Ax = b$ is equivalent at 
$$\text{solve } Qy = b$$
and then $$ \text{solve } Rx = y$$

Since $R$ is upper diagonal, solving the first problem is easy, and because $Q$ is orthonormal, solving the second problem is also easy.

First, we implement a solver for $Rx = b$, $R$ being upper triangular.

In [ ]:
def solver_TS(R, b):
    """
    Input : an invertible upper triangular matrix R, a vector b
    Output : the vector x such that Rx = b
    """
    N = len(R)
    x = np.zeros(N)
    for i in range(N-1, -1, -1):
        tmp = b[i]
        for j in range(i+1, N):
            tmp -= x[j] * R[i,j]
        x[i] = tmp / R[i,i]

    return x

Then, we implement the solver for the original problem.

In [ ]:
def solver(A, b):
    """
    Input : a square matrix A, invertible, a vector b
    Output : the vector x such that Rx = b
    """
    Q, R = QR_decomp(A)
    y = np.transpose(Q) @ b
    x = solver_TS(R, y)
    return x

## Problem 4 : Least squares problem
In this problem, we implement a least squares method to solve $Ax=b$, with $A$ a rectangular matrix.

The least squares problem can be formulate as follow :

> Find $\bar{x}$ such that $$\forall y, \Vert A\bar{x} - b \Vert \leq \Vert Ay - b\Vert$$

We know (see example 2.17 of the book) that such $\bar{x}$ is given by :
$$A^T A \bar{x} = A^Tb$$
where $A^TA$ is a square invertible matrix (as soon as $A$ has full rank). Then, we can use Problem 3 to solve those normal equations.

In [ ]:
def least_squares_pb(A, b):
    """
    Input : A rectangular matrix A, which has full rank, a vector b
    Output : minimizer x of ||A x - b ||
    """
    A_t = np.transpose(A)
    x = solver(A_t @ A, A_t @ b)
    return x

## Problem 5 : QR eigenvalue algorithm
I propose here the basic version of the QR algorithm (no improvements using Hessenberg matrix or shift). This version is the slowest, but as efficient as the others.

In [ ]:
def QR_eigenpair(A, m):
    """
    Input : a square matrix A, a number of iteration m
    Output : m-th iteration of the QR algorithm
    """
    U = np.eye(len(A))
    T = deepcopy(A)
    for _ in range(m):
        Q, R = QR_decomp(T)
        T = R @ Q
        U = U @ Q

    return T, U

## Problem 6 : Blocked matrix-matrix product
We implement a blocked matrix-matrix product, giving the possibility to the user to choose the block sizes.

In [ ]:
def blocked_matrix_matrix_product(A, B, blockA, blockB):
    """
    Input : 2 matrices A and B, a list of block size for A and B,
        under the form [[(nbr_row, nbr_col), (,),..], [(,), ...(,)]]
    Output : result of A@B
    """
    m, _ = A.shape
    _, p = B.shape
    C = np.zeros((m, p))

    nbr_block_A = blockA.shape
    nbr_block_B = blockB.shape

    ind_row_C = 0 # to know in which row of C (and A) we are

    for i in range(nbr_block_A[0]):
        ind_col_C = 0 # to know in which col of C (and B) we are
        for j in range(nbr_block_B[1]):
            ind_row_B = 0
            ind_col_A = 0
            for k in range(nbr_block_A[1]):
                ind_row_C_end = ind_row_C + blockA[i,k][0]
                ind_col_C_end = ind_col_C + blockB[k,j][1]
                ind_col_A_end = ind_col_A + blockA[i,k][1]
                ind_row_B_end = ind_row_B + blockB[k,j][0]

                block = A[ind_row_C:ind_row_C_end, ind_col_A:ind_col_A_end] @ B[ind_row_B:ind_row_B_end, ind_col_C:ind_col_C_end]                
                C[ind_row_C:ind_row_C_end, ind_col_C:ind_col_C_end] += block

                ind_col_A = ind_col_A_end
                ind_row_B = ind_row_B_end
            
            ind_col_C = ind_col_C_end
        ind_row_C = ind_row_C_end
    return C

# **Discussion**

## Problem 1 : test
To test our programm, we create matrices of the form $$Tridiag(-1,2,-1) $$ and compare the computational time and the result between a dense matrix-vector multiplication, and a sparse matrix-vector multiplication for several sizes.

In [ ]:
def create_tridiag(size):
    d = np.diag([2 for _ in range(size)], 0)
    sd = [-1 for _ in range(size-1)]
    ud = np.diag(sd, 1)
    ld = np.diag(sd, 1)
    return d + ud + ld


sizes = [10, 100, 500, 1000]
time_np = [0,0,0,0]
time_sp = [0,0,0,0]
accuracy = [0,0,0,0]

for ind, size in enumerate(sizes):
    vec = np.ones(size)
    T = create_tridiag(size)
    T_sp = Sparse()
    T_sp.np_to_sp(T)

    start_np = time.time()
    rslt_np = T@vec
    time_np[ind] = time.time() - start_np

    start_sp = time.time()
    rslt_sp = sp_matrix_vector(T_sp, vec)
    time_np[ind] = time.time() - start_sp

    accuracy[ind] = np.linalg.norm(rslt_sp - rslt_np)


plt.figure(2)
plt.plot(sizes, accuracy, '-x')
plt.xlabel("Size of matrix")
plt.ylabel("Difference between the result of \n dense and sparse multiplication")

plt.figure(1)
plt.plot(sizes, time_np, 'x-')
plt.plot(sizes, time_sp, 'x-')
plt.legend(["Time for dense matrix", "Time for sparse matrix"])
plt.xlabel("Size of matrix")
plt.ylabel("Time (s)")

We can see that the sparse matrix-vector multiplication is well implemented, and that its computational time remains almost constant while the size is increasing, which is not the case for dense matrix. 

## Problem 2 : test
To test if the QR factorization is correct, we can check the properties of matrices $Q$ and $R$ ($Q$ must be orthonormal, $R$ upper triangular) and that $QR = A$. We will use Frobenius norm for those test.

In [ ]:
n = 10
A = np.random.random((n,n))
Q, R = QR_decomp(A)

## Test if R is upper triangular ##
sum = 0
for i in range(1,n):
    for j in range(0, i):
        sum += abs(R[i,j])
print("Sum of the absolute value of lower part of R : ", sum)

## Test orthonormality of Q ##
print("Frobenius norm of Q^T @ Q - I : ", np.linalg.norm(np.transpose(Q)@Q - np.eye(n), 'fro'))

## Test decomposition of A = QR ##
print("Frobenius norm of A - QR : ", np.linalg.norm(A - Q@R, 'fro'))

All the tests return expected results. 

## Problem 3 : test
We test our solver for linear quadratic problem $Ax = b$ by two different way. First we compute $$\Vert Ax - b \Vert $$ and secondly we compute $$\Vert x - y \Vert $$ where $y$ is a manufactured solution of the problem.

In [ ]:
n = 10
A = np.random.random((n,n))
b = np.random.random(n)

x = solver(A, b)
y = np.linalg.solve(A, b)
print("Norm of Ax - b : ", np.linalg.norm(A@x - b))
print("Norm of x - y : ", np.linalg.norm(x-y))


The difference between $Ax$ and $b$, or between $x$ and $y$ almost reached the machine precision, so we can consider our solution as exact.

## Problem 4 : test
To test our implementation of the least square problem, we compute the norm of the residual :$$ \Vert r \Vert = \Vert Ax - b\Vert $$
and we compare it with others norm $$\Vert Ay-b\Vert$$ for random vectors $y$, in order to test the argmin property of $x$.

In [ ]:
n, m = 20, 10
A = np.random.rand(n,m)
b = np.random.rand(n)

x = least_squares_pb(A, b)
r = np.linalg.norm(A@x - b)
print("Norm of Ax - b : ", r)

test = 1
for _ in range(1000):
    y = np.random.random(m)
    if np.linalg.norm(A@y-b) < r:
        test = 0
        break

if not test:
    print("We have find a better approximation; the implemntation is incorrect")
    print(np.linalg.norm(A@y-b))
    print(np.linalg.det(A@np.transpose(A)))
else:
    print("We did not find a better approximation.")




## Problem 5 : test
We can test the accuracy of the eigenpairs given by the QR algorithm by computing $\det(A-\lambda_i I)$ (should be equal at zero) and the norm of $\Vert A v_i - \lambda_i v_i\Vert$ (should be 0 too).

In [ ]:
A = np.random.rand(10,10)
T, U = QR_eigenpair(A, 200)

dets = []
norms = []
for i in range(10):
    dets.append(np.linalg.det(A-T[i,i]*np.eye(10)))
    norms.append(np.linalg.norm(A@U[:,i]-T[i,i]*U[:,i]))

print("Determinants : ", dets)
print("Norms : ", norms)


We can see that after 200 iterations, all the eigenvalues have been found with a precision of $10^{-2}$. The convergence towards the eigenvectors is slowest. As said above, this algorithm can be highly improved using shift and Hessenberg reductions.

## Problem 6 : test
We test our algorithm on a particular example, where both matrices A and B have been divided into 4 blocks (2 in row, 2 in columns).

In [ ]:
A = np.reshape(np.arange(0, 21), (7,3))
B = np.reshape(np.arange(5,29), (3, 8))

blockA = np.array([[(2,2), (2,1)], [(5,2), (5,1)]])
blockB = np.array([[(2,5), (2,3)], [(1,5), (1,3)]])

C = blocked_matrix_matrix_product(A, B, blockA, blockB)

print("DIfference between normal and blocked multiplication : ", np.linalg.norm(C - A @ B))

The difference between our implementation and the normal multiplication is null, so we can be confident in our work.

# Conclusion

All the problems have been succesfully implemented and tested. 
